In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re
import os

In [4]:
pwd = os.getcwd()
file_path = os.path.join('Resources', 'Sentiment.csv')
data = pd.read_csv(file_path)

# Read in csv file
data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [5]:
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [7]:
# We will create a function to remove unwanted characters in Tweets using Regex.
# first will lower case everything, then substitue everything not within the numerical or alphabet with space
# next it will remove the RT tags (retweet)
def preProcess_data(text):
   text = text.lower()
   new_text = re.sub('[^a-zA-z0-9\s]','',text)
   new_text = re.sub('rt', '', new_text)
   return new_text

# apply the preprocessing to the text column 
data['text'] = data['text'].apply(preProcess_data)
data.head()

,text,sentiment
0,nancyleegrahn how did everyone feel about the...,Neutral
1,scottwalker didnt catch the full gopdebate la...,Positive
2,tjmshow no mention of tamir rice and the gopd...,Neutral
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive


In [8]:
# Tensorflow’s tokenizer to tokenize our dataset, and Tensorflow’s pad_sequences to pad our sequences.
max_fatures = 2000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, 28) 

# separate sentiment column and turn to values. Use this column as your target
Y = pd.get_dummies(data['sentiment']).values

In [9]:
# split and train model
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)

In [10]:
# It is now time to design and create the deep learning model. 
# We will simply use an embedding layer and some LSTM layers with dropout.
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [11]:
# fit the model
batch_size = 512

model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_data=(X_test, Y_test))

Epoch 1/10
22/22 [==============================] - 18s 669ms/step - loss: 0.9633 - accuracy: 0.6000 - val_loss: 0.8781 - val_accuracy: 0.6238
Epoch 2/10
22/22 [==============================] - 16s 726ms/step - loss: 0.8431 - accuracy: 0.6235 - val_loss: 0.7948 - val_accuracy: 0.6548
Epoch 3/10
22/22 [==============================] - 16s 737ms/step - loss: 0.7513 - accuracy: 0.6724 - val_loss: 0.7551 - val_accuracy: 0.6807
Epoch 4/10
22/22 [==============================] - 16s 741ms/step - loss: 0.6880 - accuracy: 0.6970 - val_loss: 0.7313 - val_accuracy: 0.6811
Epoch 5/10
22/22 [==============================] - 16s 744ms/step - loss: 0.6444 - accuracy: 0.7249 - val_loss: 0.7376 - val_accuracy: 0.6836
Epoch 6/10
22/22 [==============================] - 16s 745ms/step - loss: 0.6153 - accuracy: 0.7370 - val_loss: 0.7370 - val_accuracy: 0.6847
Epoch 7/10
22/22 [==============================] - 17s 756ms/step - loss: 0.5997 - accuracy: 0.7420 - val_loss: 0.7386 - val_accuracy: 0.6905

In [12]:
# save model so we don't have to run everytime
model.save('sentiment.h5')